Depositos para enviar, orderm de preferencia:
musical_matriz
musical_filal
em seguida onde houver mais estoque


onde days_available = 0 criar observação onde informa que há muito tempo sem estoque

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [2]:
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

### Período a consultar

In [3]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 11, 1).date()
data_fim = datetime(2023, 12, 8).date()
data_fim = data_fim + timedelta(days=1)  # + 1 dia para pegar a data atual no DB
print(data_fim)

2023-12-09


### Historico de estoque fulfillment

In [4]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim};'"
    # sql_query = f"SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '2023-12-04' AND '2023-12-05';"
    print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '2023-11-01' AND '2023-12-09;'


C:\Users\couti\AppData\Local\Temp\ipykernel_14708\2016158527.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [5]:
# df_stock

In [6]:
# print(df_stock.shape)
# duplicates = df_stock[df_stock.duplicated(keep=False)]
# duplicates

In [7]:
# # datas consultadas, dias em que um produto pode ou não estar disponível
# # df_stock["created_at"].value_counts().index.to_list()
# df_stock["created_at"].value_counts().sort_index()
# df_stock["created_at"].dt.date.value_counts().sort_index()
# # df_stock["created_at"].dt.date.value_counts()

# # len(df_stock["created_at"].dt.date.value_counts().sort_index())

In [8]:
# df_stock

In [9]:
# # buscando anuncio e verificando dias consultados
# result = df_stock[df_stock['ml_inventory_id'] == 'FBAY80623']
# result = result.sort_values(by='created_at')
# result

In [10]:
# # encontrando anuncios pela data
# df_stock["created_at"] = pd.to_datetime(df_stock["created_at"])
# result = df_stock.loc[df_stock['created_at'].dt.date == pd.to_datetime('2023-12-7').date()]
# result

In [11]:
# df_stock[df_stock['created_at']=='2023-12-07 03:01:03.036386']

In [12]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by="created_at", ascending=False)
df_stock["data"] = df_stock["created_at"].dt.date
df_stock = df_stock.drop(["created_at"], axis=1)

# df_stock

In [13]:
# df_stock.dtypes

In [14]:
# df_stock['data'] = pd.to_datetime(df_stock['data'])

# dfx = condf(df_stock, 'data', '2023-11-1')
# dfx

In [15]:
# # Supondo que result seja o seu DataFrame
# # A linha abaixo conta as ocorrências de cada valor em 'ml_inventory_id'
# # value_counts = result['ml_inventory_id'].value_counts()
# value_counts = df_stock['ml_inventory_id'].value_counts()

# # Filtra para incluir apenas aqueles que ocorrem mais de 2 vezes
# filtered_counts = value_counts[value_counts > 1]

# filtered_counts.shape
# # filtered_counts
# value_counts

In [16]:
# df_stock[df_stock["detail_status"] == "transfer"]

In [17]:
# condf(df_stock,'ml_inventory_id','CRMK90073')

In [18]:
# condf(df_stock, "detail_status", "transfer")

In [19]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by="data", ascending=False).reset_index(drop=True)

In [20]:
# df_stock

In [21]:
# condf(df_stock, "ml_inventory_id", "DSGP06967")

In [22]:
# print(df_stock.shape)
# df_stock = df_stock.drop_duplicates()
# print(df_stock.shape)

#### Dias em que produto esteve disponível

In [23]:
# condf(df_stock,'ml_inventory_id','ABCB20467')

In [25]:
## Contando dias em que produto esteve disponível
days_available = df_stock.groupby(["ml_inventory_id"])["has_stock"].sum().reset_index()
days_available = days_available.rename(columns={"has_stock": "days_available"})

In [27]:
# days_available.sort_values(by='days_available',ascending=False)
# days_available['days_available'].value_counts()

# ## Contando dias em que produto esteve disponível
# days_available = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available = days_available.rename(columns={"has_stock": "days_available"})
# days_available

In [28]:
# condf(df_stock, "ml_inventory_id", "MENA66694")

In [29]:
# ## Contando dias em que produto esteve disponível
# days_available_ = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available_ = days_available_.rename(columns={"has_stock": "days_available"})
# days_available_

In [30]:
# condf(days_available, "ml_inventory_id", "DSGP06967")
# condf(days_available, "ml_inventory_id", "DSGP06967")

In [31]:
# Unindo DFs
df_stock = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# df_stock.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# df_stock = df_stock.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# Verificando o formato após a correção
# print(df_stock.shape)
#

In [33]:
# df_stock

In [34]:
# # Unindo DFs
# df_stock_ = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# # df_stock_.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# # df_stock_ = df_stock_.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# # Verificando o formato após a correção
# print(df_stock_.shape)
# df_stock_

In [35]:
# data de hoje
# data_de_hoje = datetime.now().date() - timedelta(days=1)
# print(data_de_hoje)
data_de_hoje = datetime.now().date()
print(data_de_hoje)
df_stock["data"] = pd.to_datetime(df_stock["data"])
# data_de_hoje = data_de_hoje + timedelta(days=1)  # + 1 dia para pegar a data atual no DB

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock[df_stock["data"].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(
    columns={"available_quantity": "available_quantity_today"}
)
# df_stock_today = df_stock.drop(['has_stock'], axis=1)

2023-12-07


In [37]:
# df_stock_today

In [38]:
# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity

condicao = df_stock_today["detail_status"] == "transfer"

df_stock_today.loc[condicao, "available_quantity_today"] += df_stock_today.loc[
    condicao, "detail_quantity"
]

In [40]:
# df_stock_today

In [41]:
# condf(df_stock,'ml_inventory_id','FSNB97805')
# condf(df_stock_today,'ml_inventory_id','FSNB97805')

### Buscando hitorico de orders no BD

In [42]:
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM ml_orders WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# # Ordenando orders por data
# df_orders = df_orders.sort_values(by="date_approved", ascending=False)
# df_orders["data"] = df_orders["date_approved"].dt.date
# df_orders = df_orders.drop(["date_closed","date_approved"], axis=1)

# filtros
df_orders = df_orders[df_orders["logistic_type"] == "fulfillment"]
# df_orders = df_orders.drop(columns=['category_id','pack_id','variation_attributes_id','variation_name','variation_value_id', 'data'])
df_orders = df_orders.drop(
    columns=[
        "category_id",
        "pack_id",
        "variation_attributes_id",
        "variation_name",
        "variation_value_id",
    ]
)

# change column
df_orders["variation_id"] = df_orders["variation_id"].replace("nan", "0", regex=True)

# df_orders = df_orders[df_orders["order_status"] == "paid"]
# df_orders = df_orders[df_orders["payment_status"] == "approved"]


# df_orders = df_orders.drop(
#     ["pack_id", "date_approved", "order_status", "payment_status"], axis=1
# )
df_orders.rename(columns={"quantity": "sold_quantity"}, inplace=True)

SELECT * FROM ml_orders WHERE date_closed BETWEEN '2023-11-01' AND '2023-12-09'


C:\Users\couti\AppData\Local\Temp\ipykernel_14708\3275459708.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql(sql_query, conn)


In [43]:
print(df_orders.shape)
df_orders = df_orders.drop_duplicates()
print(df_orders.shape)

(5352, 13)
(4612, 13)


In [45]:
# df_orders.head()

In [46]:
# df_orders
# df_orders['pack_id'].value_counts()

# # condf(df_orders,'pack_id','2000005059931419')

In [48]:
# print(df_orders.shape)
# df_orders.head(3)

In [49]:
# condf(df_orders, "ml_code", "MLB1992541482")

#### Total de vendas por ml_code e variation_id

In [51]:
# df_orders.shape

In [52]:
# df_orders

In [53]:
# condf(df_orders,'ml_code','MLB3516894673')

In [54]:
# # Verificar se a coluna variation_id existe
# condicao_variation = df_orders["variation_id"].notna()

resultado_code_variation = (
    df_orders.groupby(["ml_code", "variation_id"])["sold_quantity"].sum().reset_index()
)

# Exibir o resultado_ml_code
# resultado_ml_code['variation_id'].value_counts()

In [56]:
# resultado_code_variation

In [57]:
# condf(resultado_code_variation,'ml_code','MLB1363602423')
# condf(df_orders,'ml_code','MLB1363602423')

# condf(resultado_code_variation,'variation_id','174301393967')
# condf(resultado_code_variation,'variation_id','174302125663')
# condf(resultado_code_variation,'ml_code','MLB1399101853')
# condf(df_orders,'ml_code','MLB1399101853')
# condf(resultado_code_variation,'ml_code','MLB4237823132')
# condf(df_orders,'ml_code','MLB4237823132')
# condf(resultado_code_variation,'ml_code','MLB950214971')
# condf(df_orders,'ml_code','MLB950214971')

In [58]:
# condf(df_orders,'variation_id','174301393967')

In [59]:
# Ordenando orders por data
df_orders = df_orders.sort_values(by="date_approved", ascending=False)
df_orders["data"] = df_orders["date_approved"].dt.date
df_orders = df_orders.drop(["date_closed", "date_approved"], axis=1)

In [60]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(
    # df_orders, resultado, on=["ml_code", "variation_id"], how="inner"
    df_orders,
    resultado_code_variation,
    on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales = df_total_sales.rename(
    columns={"sold_quantity_y": "total_sold_quantity"}
)
df_total_sales = df_total_sales.drop(
    columns=["sold_quantity_x", "order_status", "payment_status"]
)

In [64]:
# df_total_sales.shape

In [65]:
# df_orders.shape

In [66]:
# df_total_sales.columns

In [67]:
# df_total_sales.sample()

In [68]:
# # # Exibir todas as linhas e colunas
# # pd.set_option('display.max_rows', None)
# # pd.set_option('display.max_columns', None)

# # Voltar ao normal
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')


# # condf(df_total_sales,'ml_code','MLB3427769549')
# # # x = condf(df_orders,'ml_code','MLB3427769549')

# # # x = x.sort_values(by='data')
# # # x.head(15)
# # x = df_orders[df_orders['ml_code']=='MLB3427769549']
# # x['data'] = pd.to_datetime(x['data'])
# # x[x['data']=='2023-11-22']
# # # x['sold_quantity'].value_counts()

# # # counts = x[x['sold_quantity'] > 1]['sold_quantity'].value_counts()
# # # counts

# # y = x.groupby('data')['sold_quantity'].sum().reset_index()
# # y

# # # # x

In [69]:
# x.dtypes

In [70]:
# df_total_sales['data'].value_counts()

In [71]:
# condf(df_total_sales, "ml_code", "MLB4127982748")
# condf(df_total_sales, "ml_code", "MLB4127982748")

In [72]:
# df_total_sales = df_total_sales.drop(["sold_quantity_x", "shipping_id", "data"], axis=1)
# df_total_sales = df_total_sales.drop_duplicates()

In [73]:
# condf(df_total_sales, "ml_code", "MLB1992541482")

Neste ponto temos o total de itens vendidos de um anúncio por período e a quantidade de dias em que um produto esteve disponível.
precisamos juntar esses dados para calcular, para isso trarei as informações de produtos

In [ ]:
# print(df_total_sales.shape)
# df_total_sales.sample()

In [ ]:
# condf(df_total_sales, 'ml_code', 'MLB3461718736')
# condf(df_orders, 'ml_code', 'MLB3461718736')
# # condf(resultado, 'ml_code', 'MLB3427769549')
# # condf(df_stock, 'ml_code', 'MLB3427769549')

In [ ]:
# print(df_stock_today.shape)
# df_stock_today.sample()

#### Buscando Produtos

In [74]:
# Buscando dados de produtos na tabela tiny_fulfillment
try:
    conn = psycopg2.connect(**db_config)

    sql_query = "SELECT * FROM items"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar tiny_fulfillment: {e}")

except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

# df_codes["ml_code"] = df_codes["ml_code"].apply(lambda x: "MLB" + str(x))
df_codes.rename(columns={"inventory_id": "ml_inventory_id"}, inplace=True)
df_codes = df_codes.drop(["created_at", "updated_at"], axis=1)

C:\Users\couti\AppData\Local\Temp\ipykernel_14708\2324179030.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


In [75]:
df_not_catalogo = df_codes[df_codes["catalog_listing"] == False]
df_catalogo = df_codes[df_codes["catalog_listing"] == True]

In [76]:
# df_not_catalogo.sample()

In [77]:
# df_catalogo.sample()

In [78]:
# df_orders.sample()

In [79]:
df_total_sales_cat = pd.merge(
    df_catalogo,
    df_total_sales,
    left_on=["ml_code"],
    right_on=["ml_code"],
    how="inner",
)
df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_not_cat = pd.merge(
    df_not_catalogo,
    df_total_sales,
    left_on=["ml_code", "variation_id"],
    right_on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates()

In [80]:
print(df_total_sales_cat.shape)
df_total_sales_cat = df_total_sales_cat.drop(
    columns=["data", "shipping_id", "variation_id_x", "order_id"]
)
df_total_sales_cat = df_total_sales_cat.drop_duplicates()
print(df_total_sales_cat.shape)

(2681, 15)
(119, 11)


In [ ]:
# df_total_sales_cat['value_name'].value_counts()

In [81]:
# condf(df_total_sales_cat, 'ml_code','MLB3427769549')

In [82]:
# df_vendas_total_sales_not_cat

In [83]:
print(df_total_sales_not_cat.shape)
df_total_sales_not_cat = df_total_sales_not_cat.drop(
    columns=["data", "shipping_id", "order_id"]
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates()
print(df_total_sales_not_cat.shape)

(1385, 14)
(115, 11)


In [84]:
# df_total_sales_cat['ml_inventory_id'].value_counts()

In [85]:
# Antes de desabilitar
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# Seu código para exibir o DataFrame

# # Depois de desabilitar
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')


# df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_cat = df_total_sales_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)

In [86]:
# condf(df_total_sales_cat,'ml_inventory_id','WTHI04163')
# condf(y,'ml_inventory_id','WTHI04163')

In [87]:
# condf(df_total_sales_cat,'ml_inventory_id','AWAZ99732')

## Até aqui ok

In [88]:
df_total_cat = df_total_sales_cat.copy()
df_total_not_cat = df_total_sales_not_cat.copy()

In [89]:
# Somando total de vendas por inventory_id
df_sum_qt_sold_cat = (
    df_total_cat.groupby("ml_inventory_id")["total_sold_quantity"].sum().reset_index()
)
df_sum_qt_sold_cat = df_sum_qt_sold_cat.rename(
    columns={"total_sold_quantity": "total_sold_catalog"}
)

df_total_cat = pd.merge(
    df_total_cat, df_sum_qt_sold_cat, on="ml_inventory_id", how="left"
)

df_total_cat.rename(columns={"variation_id_y": "variation_id_"})


df_sum_qt_sold_not_cat = (
    df_total_not_cat.groupby("ml_inventory_id")["total_sold_quantity"]
    .sum()
    .reset_index()
)
df_sum_qt_sold_not_cat = df_sum_qt_sold_not_cat.rename(
    columns={"total_sold_quantity": "total_sold_not_catalog"}
)
df_total_not_cat = pd.merge(
    df_total_not_cat, df_sum_qt_sold_not_cat, on="ml_inventory_id", how="left"
)

In [90]:
df_total_cat.rename(columns={"variation_id_y": "variation_id"}, inplace=True)
# df_total_cat.sample(2)

In [ ]:
# condf(df_total_cat,'ml_inventory_id','WTHI04163')

In [91]:
df_total_cat = df_total_cat.drop_duplicates(subset=["ml_inventory_id"])

# df_total_cat[df_total_cat['ml_inventory_id']=='WTHI04163']

pergar ml_inventory_id e sum_total e acrescesntar essas informações no df de não catalogo pelo ml_inventory_id
ao fim deixar somatório do catalogo e não catalogo


In [92]:
# cols=['ml_inventory_id','total_sold_catalog']
# df_total_cat = df_total_cat[cols]
# df_total_cat.sample()

In [93]:
df_combined = pd.merge(
    df_total_not_cat,
    df_total_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)
# df_combined.sample()

In [94]:
# df_total_not_cat.shape

In [95]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [96]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [97]:
# df_combined['ml_inventory_id'].value_counts()

In [98]:
df = pd.merge(df_combined, df_stock_today, on="ml_inventory_id", how="inner")
df["total_sold_catalog"] = df["total_sold_catalog"].fillna(0).astype("int64")

In [99]:
# df.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,seller_sku,title,variation_value_name,logistic_type,...,total_sold_not_catalog,total_sold_catalog,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
49,MLB2075516288,FHRY37807,6 PARES 7A,173887022579,active,False,FULLLIVERPOOL6NI7AM,Kit 6 Pares De Baquetas D Bateria Liverpool Ni...,6 PARES 7A,fulfillment,...,23,0,15,None,NaN,MLB2075516288,173887022579,2023-12-07,True,36


In [100]:
days = 30

df["total_sold"] = df["total_sold_catalog"] + df["total_sold_not_catalog"]
# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df["period_send_fulfillment"] = np.ceil(
    (df["total_sold"] / df["days_available"]) * days - df["available_quantity_today"]
)

df["period_send_fulfillment"] = df["period_send_fulfillment"].fillna(0)

In [101]:
df_have_itens = df[df["days_available"] > 0]
df_have_itens.sample(2)

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,seller_sku,title,variation_value_name,logistic_type,...,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available,total_sold,period_send_fulfillment
25,MLB1992541482,CLMF99895,0.12,173552533267,active,False,FULLSG6687,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,0.12,fulfillment,...,11,lost,2.0,MLB1992541482,173552533267,2023-12-07,True,36,20,6.0
10,MLB1478212260,ENML56386,Preto/Amarelo,52917805648,active,False,FULLHERCULESGS302B,Suporte Profissi Travlite Hércules P/ Guitarra...,Preto/Amarelo,fulfillment,...,2,None,NaN,MLB1478212260,52917805648,2023-12-07,True,36,3,1.0


In [102]:
# produtos sem estoque no período
df_no_itens = df[df["days_available"] <= 0]
df_no_itens = df_no_itens.drop(columns=["period_send_fulfillment"])
df_no_itens.sample(2)

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,seller_sku,title,variation_value_name,logistic_type,...,total_sold_catalog,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available,total_sold
93,MLB2986538160,PPWL59058,Vermelho,175887797708,active,False,FULLLIVERPOOLPML001V,Pandeiro Profissional Meia Lua Diversas Cores ...,Vermelho,fulfillment,...,21,0,lost,1.0,MLB2986538160,175887797708,2023-12-07,False,0,24
14,MLB1662816343,GDCT87609,Vermelho,64488024031,paused,False,FULLIZZO6493,Pandeiro Original 10 Pol Abs Vermelho C/pele M...,Vermelho,fulfillment,...,0,0,None,NaN,MLB1662816343,64488024031,2023-12-07,False,0,1


In [103]:
# df_have_itens.sample()

In [104]:
dfx = df_have_itens.copy()

In [105]:
# dfx.columns

In [106]:
# # Exibir todas as linhas e colunas
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# Voltar ao normal
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

# dfx = dfx.drop(columns=['catalog_listing','variation_value_name','variation_id','logistic_type','references_id','has_stock','total_sold_quantity',
#                         'references_variation_id'])

cols = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "days_available",
    "total_sold_not_catalog",
    "total_sold_catalog",
    "total_sold",
    "period_send_fulfillment",
]

dfx = dfx[cols]
df.rename(
    columns={"detail_status": "transfer_status", "detail_quantity": "transfer_quantity"}
)
# dfx.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,seller_sku,title,variation_value_name,logistic_type,...,available_quantity_today,transfer_status,transfer_quantity,references_id,references_variation_id,data,has_stock,days_available,total_sold,period_send_fulfillment
0,MLB1398114866,DXJZ75064,0.010,174302125663,active,False,FULLELIXIR12052,Encordoamento Guitarra Elixir Nanoweb Original...,0.010,fulfillment,...,1,None,NaN,MLB1398114866,174302125663,2023-12-07,True,36,2,1.0
1,MLB1398114866,JDMV76187,0.009,174301393967,active,False,FULLELIXIR12002,Encordoamento Guitarra Elixir Nanoweb Original...,0.009,fulfillment,...,2,None,NaN,MLB1398114866,174301393967,2023-12-07,True,36,1,-1.0
2,MLB1418532096,SCVF31712,Preto/Madeira,49773918421,active,False,FULLHERCULESGSP38WB,Suporte P/ Guitarra De Parede Com Trava Ags He...,Preto/Madeira,fulfillment,...,36,None,NaN,MLB1418532096,49773918421,2023-12-07,True,36,3,-33.0
3,MLB1425113746,XDTG30724,Preto,50098452016,active,False,FULLHERCULESGSP39WB,Suporte Guitarra Baixo Violão Para Parede Herc...,Preto,fulfillment,...,1,lost,1.0,MLB1425113746,50098452016,2023-12-07,True,36,58,48.0
4,MLB1461203307,CNHG47042,Marrom,51909128584,paused,False,FULLKALANI15421,Ukulele Concerto 24 Kayke Mogno Kal 320 Cm Com...,Marrom,fulfillment,...,0,None,NaN,MLB1461203307,51909128584,2023-12-07,False,0,6,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,MLB3339660079,ARDK78657,2,178533059697,active,False,FULLVANDOREN2SR212,"2 Palheta Sax Alto Vandoren Tradicional 1 1,5 ...",2,fulfillment,...,15,None,NaN,MLB3339660079,178533059697,2023-12-07,True,36,13,-4.0
111,MLB3339660079,IDSV51539,"2,5",178533059699,active,False,FULLVANDOREN2SR2125,"2 Palheta Sax Alto Vandoren Tradicional 1 1,5 ...","2,5",fulfillment,...,64,None,NaN,MLB3339660079,178533059699,2023-12-07,True,36,4,-60.0
112,MLB3339660079,BZGW51616,"1,5",178533059701,active,False,FULLVANDOREN2SR2115,"2 Palheta Sax Alto Vandoren Tradicional 1 1,5 ...","1,5",fulfillment,...,25,None,NaN,MLB3339660079,178533059701,2023-12-07,True,36,10,-16.0
113,MLB1662970458,UIUW89800,Preto,64498487033,active,False,FULLIZZO6480,Pandeiro Original 10 Pol Abs Preto Com Pele Pr...,Preto,fulfillment,...,12,transfer,12.0,MLB1662970458,64498487033,2023-12-07,False,0,581,inf


## Acabou!

In [ ]:
# # media de produtos disponiveis no período
# df_vendas_total_sales_not_cat["media_prod_days_available"] = (
#     df_vendas_total_sales_not_cat["total_sold_quantity"] / df_vendas_total_sales_not_cat["days_available"]
# )
# df_vendas_total_sales_not_cat["media_prod_days_available"] = df_vendas_total_sales_not_cat["media_prod_days_available"].fillna(0)

# days = 30

# # qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
# df_vendas_total_sales_not_cat["period_send_fulfillment"] = np.ceil(
#     (df_vendas_total_sales_not_cat["total_sold_quantity"] / df_vendas_total_sales_not_cat["days_available"]) * days
#     - df_vendas_total_sales_not_cat["available_quantity_today"]
# )
# df_vendas_total_sales_not_cat["period_send_fulfillment"] = df_vendas_total_sales_not_cat["period_send_fulfillment"].fillna(0)

# # qtd de produtos a enviar hoje, caso seja valor negativo produto está acima do esperado para envio(sobrando)
# df_vendas_total_sales_not_cat["today_send_fulfillment"] = np.ceil(
#     (df_vendas_total_sales_not_cat["total_sold_quantity"] / df_vendas_total_sales_not_cat["days_available"])
#     - df_vendas_total_sales_not_cat["available_quantity_today"]
# )
# df_vendas_total_sales_not_cat["today_send_fulfillment"] = df_vendas_total_sales_not_cat["today_send_fulfillment"].fillna(0)

In [ ]:
# resultado_cat = (
#     df_vendas_cat.groupby("ml_inventory_id")["sold_quantity"].sum().reset_index()
# )

# resultado_cat

In [ ]:
# resultado_not_cat = (
#     df_vendas_not_cat.groupby("ml_inventory_id")["sold_quantity"].sum().reset_index()
# )
# resultado_not_cat

In [ ]:
# resultado = pd.merge(
#     resultado_cat, resultado_not_cat, on=["ml_inventory_id"], how="outer"
# )
# resultado = resultado.fillna(0)
# # pd.set_option('display.max_rows', None)
# pd.reset_option("display.max_rows")

# resultado["sold_quantity_sum"] = resultado["sold_quantity_x"].fillna(0) + resultado[
#     "sold_quantity_y"
# ].fillna(0)
# df_total_sales = resultado.copy()

### Produtos + Dias disponíveis

In [ ]:
# print(df_codes.shape)
# print(df_stock_today.shape)

# print(df_codes.columns)
# print(df_stock_today.columns)

In [ ]:
# condf(prod_day, 'ml_code', 'MLB2003190501')

In [ ]:
prod_day = pd.merge(df_codes, df_stock_today, on="ml_inventory_id", how="inner")
# prod_day = pd.merge(df_codes, df_stock_today, on="ml_inventory_id", how="outer")
prod_day = prod_day[prod_day["catalog_listing"] == False]
prod_day.shape
prod_day

### Prod_Day + Total_sales

In [ ]:
# print(df_total_sales.shape)
# df_total_sales = df_total_sales.drop(["sold_quantity_x", "sold_quantity_y"], axis=1)
# df_total_sales.sample()

In [ ]:
# # df_sales = pd.merge(
# #     df_total_sales,
# #     prod_day,
# #     left_on=["ml_code", "variation_id"],
# #     right_on=["ml_code", "variation_id"],
# #     how="inner",
# # )

# df_sales = pd.merge(
#     df_total_sales,
#     prod_day,
#     left_on=["ml_inventory_id"],
#     right_on=["ml_inventory_id"],
#     how="inner",
# )

# cols = [
#     "ml_code",
#     "ml_inventory_id",
#     "sold_quantity_sum",
#     "value_name",
#     "variation_id",
#     "status",
#     "catalog_listing",
#     "available_quantity_today",
#     "detail_status",
#     "detail_quantity",
#     "references_id",
#     "references_variation_id",
#     "data",
#     "has_stock",
#     "days_available",
# ]

# # df_sales = df_sales[cols]
# # df_sales = df_sales.rename(columns={'days_available_x':'days_available'})
# print(df_total_sales.shape)
# print(prod_day.shape)
# # print(x.shape)
# print(df_sales.shape)

In [ ]:
# df_sales.columns

In [ ]:
# df_not_sales = pd.merge(
#     df_total_sales,
#     prod_day,
#     left_on=["ml_inventory_id"],
#     right_on=["ml_inventory_id"],
#     how="inner",
# )
# # df_not_sales = df_not_sales[df_not_sales['days_available']==0]

# cols = [
#     "ml_code",
#     "ml_inventory_id",
#     "sold_quantity_sum",
#     "value_name",
#     "variation_id",
#     "status",
#     "catalog_listing",
#     "available_quantity_today",
#     "detail_status",
#     "detail_quantity",
#     "references_id",
#     "references_variation_id",
#     "data",
#     "has_stock",
#     "days_available",
# ]

# df_not_sales = df_not_sales[cols]
# # df_not_sales = df_not_sales.rename(columns={'days_available_x':'days_available'})
# print(df_total_sales.shape)
# print(prod_day.shape)
# # print(x.shape)
# print(df_not_sales.shape)

In [ ]:
# # df_not_sales
# df_sales.shape

In [ ]:
# condf(df_sales, "ml_code", "MLB1992541482")

In [ ]:
# x = pd.merge(x, df_sales, how='outer', indicator=True)
# x = x[x['_merge'] == 'right_only']

# x

### Calculando métricas

In [ ]:
x = df_sales[df_sales["ml_inventory_id"] == "AGDL67547"]
x

In [ ]:
df_sales.sample(10)

In [ ]:
# media de produtos disponiveis no período
df_sales["media_prod_days_available"] = (
    df_sales["sold_quantity_sum"] / df_sales["days_available"]
)
df_sales["media_prod_days_available"] = df_sales["media_prod_days_available"].fillna(0)

days = 30

# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df_sales["period_send_fulfillment"] = np.ceil(
    (df_sales["sold_quantity_sum"] / df_sales["days_available"]) * days
    - df_sales["available_quantity_today"]
)
df_sales["period_send_fulfillment"] = df_sales["period_send_fulfillment"].fillna(0)

# qtd de produtos a enviar hoje, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df_sales["today_send_fulfillment"] = np.ceil(
    (df_sales["sold_quantity_sum"] / df_sales["days_available"])
    - df_sales["available_quantity_today"]
)
df_sales["today_send_fulfillment"] = df_sales["today_send_fulfillment"].fillna(0)

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
import pandas as pd

# DataFrame da esquerda
df_esquerda = pd.DataFrame({"ID": [1, 2, 3, 4], "Valor_esquerda": ["A", "B", "C", "D"]})

# DataFrame da direita
df_direita = pd.DataFrame({"ID": [3, 4, 5, 6], "Valor_direita": ["X", "Y", "Z", "W"]})

# Realizando a junção anti-esquerda
resultado = pd.merge(
    df_esquerda, df_direita, on="ID", how="left", indicator=True
).query('_merge == "left_only"')

# Removendo a coluna de indicador (_merge)
resultado = resultado.drop(columns=["_merge"])

print(resultado)

In [ ]:
cols = [
    "ml_code",
    "ml_inventory_id",
    "value_name",
    "status",
    "available_quantity_today",
    "sold_quantity_sum",
    "days_available",
    "media_prod_days_available",
    "period_send_fulfillment",
    "today_send_fulfillment",
]
df_sales = df_sales[cols]
df_sales

In [ ]:
df_sales["days_available"].value_counts()

In [ ]:
df_sales.sample(2)

In [ ]:
x = "MLB1629379059"
y = df_sales[df_sales["ml_code"] == x]
y

In [ ]:
condf(df_sales, "ml_code", "MLB1992541482")

Pergunta:

Caso em seja necessário enviar produtos de um kit e apenas um dos produtos estiver em falta, o que fazer?